# Count-Based FairDeDup (CB-FDD)
This file contains our implementation of CB-FDD, the main contribution of our project.

## Data Loading and Processing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# IMPORTANT: You may need to modify the file paths to match where you stored these files.
import pickle

file_path = '/content/drive/My Drive/final_train_df.pkl'

with open(file_path, 'rb') as f:
  train_df = pickle.load(f)

train_df

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices,gender,insurance,anchor_age,race
0,0.859503,-1.287299,1.058663,-0.943677,0.706466,0.230673,0.244263,0.239631,0.300267,0.820084,...,0.0,0.0,0.0,0.0,0.0,0.0,F,Medicare,64.0,WHITE
1,-0.290506,-0.298115,1.247925,-2.139753,0.106529,1.040990,0.186258,-0.124659,-0.192760,0.619054,...,0.0,0.0,0.0,0.0,0.0,0.0,F,Medicare,64.0,WHITE
2,-0.052353,-0.349717,1.297147,-2.643505,0.269400,1.493779,0.105843,0.372665,-0.534648,0.557481,...,0.0,0.0,0.0,0.0,0.0,0.0,F,Medicare,64.0,WHITE
3,0.682515,-1.924162,1.982731,-1.724569,0.670782,-0.527549,0.527647,0.758055,-0.715179,0.472782,...,0.0,0.0,0.0,0.0,0.0,0.0,F,Other,55.0,WHITE
4,-1.377771,-2.006497,0.234329,-1.769763,0.045857,0.366333,-0.282138,-0.444843,0.871546,0.858632,...,0.0,0.0,0.0,0.0,1.0,1.0,F,Other,55.0,WHITE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21586,0.106283,-1.216278,1.067247,-1.719202,-0.570664,-0.533635,-0.073067,0.697008,0.770453,0.820607,...,0.0,0.0,0.0,0.0,0.0,0.0,F,Medicare,80.0,WHITE
21587,0.366951,-0.572345,0.601550,-1.369258,0.306596,-0.174169,-0.205630,0.068738,-0.335461,1.384266,...,1.0,0.0,0.0,0.0,0.0,0.0,F,Medicare,80.0,WHITE
21588,-0.228984,-2.159328,0.930693,-0.954802,-0.174791,0.924886,0.286401,0.013156,-0.266270,1.908059,...,1.0,0.0,0.0,0.0,0.0,0.0,M,Other,67.0,WHITE
21589,-0.507701,-0.733408,1.157441,-1.994942,0.408057,-0.026694,0.384053,-0.571007,-0.566449,0.589113,...,0.0,0.0,0.0,0.0,0.0,0.0,M,Other,76.0,WHITE


In [ ]:
# Extra pre-processing: conversion from numeric age into a categorical age range
# Same labels as A1
import pandas as pd

bins = [0, 20, 40, 60, 80, float('inf')]
labels = ['0-20', '20-40', '40-60', '60-80', '80+']

train_df['age_decile'] = pd.cut(train_df['anchor_age'], bins=bins, labels=labels)

In [ ]:
# Plus, we should also save the original indices.
train_df['og_index'] = train_df.index
train_df

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,Pleural Other,Pneumonia,Pneumothorax,Support Devices,gender,insurance,anchor_age,race,age_decile,og_index
0,0.125711,-1.803007,1.284251,-1.808832,0.127820,-0.191196,0.609272,0.830579,-0.443789,1.238895,...,0.0,0.0,0.0,0.0,F,Medicaid,52.0,WHITE,40-60,0
1,0.189455,-1.818042,1.320364,-2.396976,0.120655,-0.262529,0.738478,0.056799,-0.246806,0.450312,...,0.0,0.0,0.0,0.0,F,Medicaid,52.0,WHITE,40-60,1
2,-0.466300,-1.172919,0.114043,-1.868590,0.553166,0.417780,0.734430,0.003140,-0.292678,0.740896,...,0.0,0.0,0.0,0.0,F,Medicaid,52.0,WHITE,40-60,2
3,-0.507073,-1.562280,0.377472,-1.541743,0.293578,0.306945,0.647620,-0.074701,0.160979,0.329081,...,0.0,0.0,0.0,0.0,F,Medicaid,52.0,WHITE,40-60,3
4,-0.200524,-1.818057,0.725834,-2.209781,0.394925,-0.662887,0.430449,0.138520,0.443141,1.002451,...,0.0,0.0,0.0,0.0,F,Medicaid,52.0,WHITE,40-60,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207309,0.084385,-1.828303,0.807940,-0.793703,0.613876,-0.215462,0.361803,-0.010056,0.053416,1.002149,...,0.0,0.0,0.0,0.0,F,Other,19.0,WHITE,0-20,207309
207310,-0.327731,-2.051952,1.055242,-0.676235,0.851157,-0.242834,0.080449,0.220710,-0.300547,1.352934,...,0.0,0.0,0.0,0.0,F,Other,19.0,WHITE,0-20,207310
207311,0.593066,-1.939155,0.648141,-2.380389,0.197016,0.044840,-0.131458,0.692708,-0.744774,1.471385,...,0.0,0.0,0.0,0.0,F,Other,57.0,UNKNOWN,40-60,207311
207312,-0.033412,-1.678429,0.760117,-2.032235,-0.698559,-0.458091,0.326971,0.787460,0.682498,-0.608661,...,0.0,0.0,0.0,1.0,F,Other,57.0,UNKNOWN,40-60,207312


In [ ]:
# Pickle the updated file into 'file_train_df_new.pkl'
train_df.drop('anchor_age', axis=1, inplace=True) # gotta get rid of the old, numeric age column

file_path = '/content/drive/My Drive/final_train_df_new.pkl'

with open(file_path, 'wb') as f:
  pickle.dump(train_df, f)

In [ ]:
# Select columns of embedding only (recall each embedding vector was a size of 1376 values)

train_df_embedding_columns = train_df.iloc[:, 0:1376]
print(train_df_embedding_columns.shape)
print(train_df_embedding_columns.columns)

(207314, 1376)
Index(['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       ...
       'feature_1366', 'feature_1367', 'feature_1368', 'feature_1369',
       'feature_1370', 'feature_1371', 'feature_1372', 'feature_1373',
       'feature_1374', 'feature_1375'],
      dtype='object', length=1376)


In [ ]:
# Merge 1376 columns into 1 vector embedding for each row
all_embeddings = train_df_embedding_columns.values
print(len(all_embeddings))
print(all_embeddings)
print(all_embeddings[0].shape)

207314
[[ 0.12571079 -1.8030074   1.2842511  ... -0.7076767   1.0859656
   0.02559637]
 [ 0.18945463 -1.818042    1.3203645  ... -1.3234656   1.4244304
  -0.26066294]
 [-0.4663002  -1.1729188   0.11404295 ... -1.080736    0.6743026
  -0.33340642]
 ...
 [ 0.5930662  -1.9391551   0.6481414  ... -1.0502896  -0.48815688
  -1.756902  ]
 [-0.03341227 -1.678429    0.76011676 ... -1.6180495   0.64806
  -1.2408257 ]
 [ 0.9761496  -2.2109149   0.61055285 ... -1.4995803   0.6610715
  -1.7800003 ]]
(1376,)


## Helper Functions for CB-FDD
These functions help:
- Create a count dictionary for all sensitive groups
- Create a score dictionary for demographic combinations
- Calculate the cosine similarity between embeddings.
- Retrieve the most underrepresented point in a duplicate neighbourhood.

### Load in the clusters we got using some clustering method (Faiss, K-Means, or Mini-Batch K-Means)

In [ ]:
# with open('/content/drive/My Drive/cluster_assignments_faiss','rb') as f: cluster_assignments = pickle.load(f)

In [ ]:
# with open('/content/drive/My Drive/cluster_assignments_kmeans','rb') as f: cluster_assignments = pickle.load(f)

In [ ]:
# with open('/content/drive/My Drive/cluster_assignments_minibatch','rb') as f: cluster_assignments = pickle.load(f)

In [ ]:
# with open('/content/drive/My Drive/cluster_assignments_kmeans_500','rb') as f: cluster_assignments = pickle.load(f)

In [ ]:
with open('/content/drive/My Drive/cluster_assignments_kmeans_1500','rb') as f: cluster_assignments = pickle.load(f)

In [ ]:
cluster_assignments

array([1134, 1134, 1153, ...,  773,    4,  717], dtype=int32)

### Generate a dictionary of counts for all sensitive groups
e.g., (`male:0`, `female:1`, ...)

In [ ]:
def create_sensitive_group_dict():
  labels_gender = train_df['gender'].unique().tolist()
  labels_race = train_df['race'].unique().tolist()
  labels_age = train_df['age_decile'].unique().tolist()

  sensitive_groups_labels = labels_gender + labels_race + labels_age

  sensitive_group_dict = {group: 0 for group in sensitive_groups_labels}
  return sensitive_group_dict

sensitive_group_dict = create_sensitive_group_dict()

### Generate a score dictionary for all demographic combinations (initialized to 0)

e.g., if `male = 1`, `black = 1` ---> `score (M/B) = 2`, `(F/B) = 1`, `(F/W) = 0`

Assign each image a score using the dictionary:  
image 1 `MB` --> 2  
image 2 `FB` --> 1  
image 3 `FW` --> 0  

We choose the image with smallest score (least represented).

In [ ]:
from itertools import product

def create_score_dict():
  labels_gender = train_df['gender'].unique().tolist()
  labels_race = train_df['race'].unique().tolist()
  labels_age = train_df['age_decile'].unique().tolist()

  combinations = list(product(labels_gender, labels_race, labels_age))

  score_dict = {combin: 0 for combin in combinations}
  return score_dict

score_dict = create_score_dict()
print(score_dict)

{('F', 'WHITE', '40-60'): 0, ('F', 'WHITE', '80+'): 0, ('F', 'WHITE', '60-80'): 0, ('F', 'WHITE', '20-40'): 0, ('F', 'WHITE', '0-20'): 0, ('F', 'BLACK/AFRICAN AMERICAN', '40-60'): 0, ('F', 'BLACK/AFRICAN AMERICAN', '80+'): 0, ('F', 'BLACK/AFRICAN AMERICAN', '60-80'): 0, ('F', 'BLACK/AFRICAN AMERICAN', '20-40'): 0, ('F', 'BLACK/AFRICAN AMERICAN', '0-20'): 0, ('F', 'OTHER', '40-60'): 0, ('F', 'OTHER', '80+'): 0, ('F', 'OTHER', '60-80'): 0, ('F', 'OTHER', '20-40'): 0, ('F', 'OTHER', '0-20'): 0, ('F', 'UNKNOWN', '40-60'): 0, ('F', 'UNKNOWN', '80+'): 0, ('F', 'UNKNOWN', '60-80'): 0, ('F', 'UNKNOWN', '20-40'): 0, ('F', 'UNKNOWN', '0-20'): 0, ('F', 'ASIAN', '40-60'): 0, ('F', 'ASIAN', '80+'): 0, ('F', 'ASIAN', '60-80'): 0, ('F', 'ASIAN', '20-40'): 0, ('F', 'ASIAN', '0-20'): 0, ('F', 'HISPANIC/LATINO', '40-60'): 0, ('F', 'HISPANIC/LATINO', '80+'): 0, ('F', 'HISPANIC/LATINO', '60-80'): 0, ('F', 'HISPANIC/LATINO', '20-40'): 0, ('F', 'HISPANIC/LATINO', '0-20'): 0, ('F', 'UNABLE TO OBTAIN', '40-60

In [ ]:
# 80 keys (demographic combinations) generated
# (2 genders) * (8 races) * (5 age ranges) = 80 combinations
print(len(score_dict.keys()))

80


### Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

# Use cosine similarity to calculate the similarity between embeddings.
# We have four options... They all seem fine to me and they values are pretty close to each other.
def calculate_sim(emb0, embs):
  # OPTION 1: Use numpy
  np_sim = embs @ emb0 / (np.linalg.norm(embs, axis=1) * np.linalg.norm(emb0))
  # print(np_sim)
  # print(np_sim.shape)

  # OPTION 2: Use sklearn
  sklearn_sim = cosine_similarity(emb0.reshape(1, -1), embs)
  # print(sklearn_sim)
  # print(sklearn_sim.shape)

  # OPTION 3: Use scipy
  scipy_sim = np.array([1 - cosine(emb0, emb) for emb in embs])
  # print(scipy_sim)
  # print(scipy_sim.shape)

  # OPTION 4: The similarity provided by FairDeDup (... it produces numbers greater than 1, idk bout this one)
  # paper_sim = embs[node] @ embs.T
  # print(paper_sim)
  # print(paper_sim.shape)

  return sklearn_sim


### Retrieve most underpresented point in the neighrbouhood

In [ ]:
from collections import defaultdict

def get_underrepresented_point(neighbours, df, sensitive_group_dict):
  score_tracker = defaultdict(int)        # Maps combinations to scores
  pt_tracker = defaultdict(list)          # Maps combinations to points

  for point in neighbours:
    # Retrieve the point's sensitive attributes
    pt_gender = df.iloc[point]['gender']
    pt_race = df.iloc[point]['race']
    pt_age = df.iloc[point]['age_decile']

    # Retrieve the scores for each sensitive attributes
    score = sensitive_group_dict[pt_gender] + sensitive_group_dict[pt_race] + sensitive_group_dict[pt_age]
    key = (pt_gender, pt_race, pt_age)

    score_tracker[key] = score
    # pt_tracker[key] = point
    pt_tracker[key].append(point)

  # Find the point with the lowest score
  min_score = float('inf')
  min_combin = None
  for point, score in score_tracker.items():
    if score < min_score:
      min_score = score
      min_combin = point


  min_points = pt_tracker[min_combin] # Retrieve list of indices/points with the min combination

  # Update the sensitive_group_dict count
  sensitive_group_dict[min_combin[0]] += 1
  sensitive_group_dict[min_combin[1]] += 1
  sensitive_group_dict[min_combin[2]] += 1

  chosen_keep_point = min_points[-1].item()               # If we have multiple points with the lowest scores, just take the last one

  return chosen_keep_point

## This is the main chunk of code for CB-FDD

In [ ]:
# Called for each cluster
def fairdedup(cluster_df, eps=0.04):

  # Assumption 1: we need to define a new dictionary for each cluster (as described in FairDeDup)
  sensitive_group_dict = create_sensitive_group_dict()
  balance = create_score_dict()

  # Convert the feature columns of the dataframe into a 2D numpy array
  # Done so we can have easy comparison between embeddings
  embedding_cols = cluster_df.iloc[:, :1376].columns
  embs = cluster_df[embedding_cols].to_numpy()

  # 1D numpy array to keep track of which nodes we've visited
  tovisit = np.ones(cluster_df.shape[0])      # Initialized to all ones, node i will be set to 0 once it has been visited

  # Keep track of the indices of nodes to keep
  keep_nodes = []
  is_first_neighbourhood=True                 # Just a flag, we just select a random node if it's the first neighbourhood we've seen because sensitive_group_dict is initialized to all 0s (and scores will be all 0s)


  while tovisit.any():
    node = np.where(tovisit)[0][0]            # Get the first non-zero index from tovisit (first unvisited node)
    emb0 = embs[node]

    sims = calculate_sim(emb0, embs)
    neighbours = np.where(sims > 1 - eps)[1]  # The indices of the neighbours

    if is_first_neighbourhood:
      # Just pick a random node to keep
      point = np.random.choice(neighbours).item()
      is_first_neighbourhood = False
    else:
      # Retrieve the node that is the least represented in this cluster
      point = get_underrepresented_point(neighbours, cluster_df, sensitive_group_dict)

    keep_nodes.append(point)                  # Save the index as a point to keep

    # Set neighbour nodes to visited
    tovisit[neighbours] = 0

  print('='*50)
  print('num nodes kept: ', len(keep_nodes))
  print('num original nodes: ', cluster_df.shape[0])

  # get the original indices to keep
  og_keep_nodes = cluster_df.iloc[keep_nodes]['og_index'].tolist()
  print('og_keep_nodes:', og_keep_nodes)

  return og_keep_nodes

For each cluster, we call our modified FDD method to deduplicate the points.

In [ ]:
# This takes around 30 min to run
import numpy as np

# Iterate through all the clusters
keep_nodes_indices = []               # A list of the row indices of nodes we are keeping
for cluster_num in np.unique(cluster_assignments):
  # Retrieve the indices of the embeddings that belong to cluster_num
  # We assume the indices are preserved
  cluster_indices = np.where(cluster_assignments == cluster_num)[0] # returnes indices of embeddings who are in cluster_num

  # Retrieve the rows
  cluster_df = train_df.iloc[cluster_indices] # based off assumption: retrieves df rows belonging to cluster based on same indices

  cluster_keep_nodes = fairdedup(cluster_df)
  keep_nodes_indices.append(cluster_keep_nodes)

Streaming output truncated to the last 5000 lines.
num nodes kept:  89
num original nodes:  163
og_keep_nodes: [645, 86782, 3391, 3394, 5353, 7754, 7755, 17108, 142123, 18012, 18006, 18186, 18184, 24712, 74288, 33066, 33204, 18186, 41812, 41808, 41878, 41887, 41880, 49188, 55762, 181094, 102230, 41812, 59556, 67909, 68586, 72269, 72269, 74283, 74288, 75070, 75380, 78679, 78681, 80325, 80340, 162163, 80343, 80347, 102256, 85622, 86267, 86271, 142123, 93630, 96902, 102046, 102236, 102256, 105135, 107808, 117963, 118269, 118567, 121362, 147830, 134787, 134783, 134784, 136622, 142121, 142112, 142134, 143626, 156831, 162164, 162172, 163378, 163378, 164355, 171563, 174043, 174521, 176599, 176607, 182643, 186338, 186366, 186376, 187888, 190188, 192726, 41697, 207238]
num nodes kept:  8
num original nodes:  223
og_keep_nodes: [19631, 203580, 144540, 158615, 41568, 48907, 205895, 73085]
num nodes kept:  93
num original nodes:  154
og_keep_nodes: [722, 151539, 1141, 5765, 159617, 114660, 9606, 7

In [ ]:
import itertools

# So... it's a list of lists
print(len(keep_nodes_indices))
print(keep_nodes_indices[-1:])

# Flatten the list of lists using itertools.chain
cluster_indices_flat = list(itertools.chain.from_iterable(keep_nodes_indices))
print(len(cluster_indices_flat))

1500
[[94846, 5638, 103333, 48051, 131174, 94846, 56801, 82087, 58651, 69025, 24197, 45690, 24130, 141632, 202776, 203881]]
106842


In [ ]:
# Create a new dataframe only containing the selected nodes
selected_df = train_df.iloc[cluster_indices_flat]
print(selected_df.shape)

(106842, 1395)


In [ ]:
selected_df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices,gender,insurance,race,age_decile,og_index
4405,-0.306411,-0.294814,0.260750,-2.488408,0.564580,-0.110443,0.327829,0.398853,1.298751,0.310222,...,1.0,0.0,0.0,0.0,1.0,M,Medicare,WHITE,60-80,4405
4415,-0.700548,-0.152789,0.280155,-3.119052,0.304011,0.184494,-0.079978,0.372956,1.130187,0.286611,...,1.0,0.0,0.0,0.0,0.0,M,Medicare,WHITE,60-80,4415
4414,-0.187798,-0.747573,0.668551,-2.190409,0.038452,0.710440,-0.356037,0.129384,1.496755,0.460582,...,0.0,0.0,0.0,0.0,1.0,M,Medicare,WHITE,60-80,4414
5292,-0.546131,-0.573763,1.136663,-3.176939,0.425183,0.332545,0.162696,0.479241,0.880332,0.441770,...,1.0,0.0,0.0,0.0,1.0,F,Medicare,WHITE,80+,5292
6072,-0.712018,-1.180359,0.851289,-1.802049,-0.518341,0.063663,-0.604475,0.885063,1.031677,0.765335,...,1.0,0.0,0.0,1.0,1.0,M,Other,WHITE,20-40,6072


In [ ]:
# Pickle it for good measure
file_path = '/content/drive/My Drive/deduplicated_train_df.pkl'

with open(file_path, 'wb') as f:
  pickle.dump(selected_df, f)

In [ ]:
# Here just for reference
# This is the pseudocode for the original FairDeDup.
'''
def fairdedup_original(embs, prototypes, eps):
  # Get similarity with concept prototypes
  proto = embs @ prototypes.T

  balance = AverageMeter(prototype.shape[0])
  tovisit = torch.ones(embs.shape[0])

  while tovisit.any():
    # Find an unvisited neighborhood
    node = torch.where(tovisit)[0][0]
    sims = embs[node] @ embs.T
    neighbors = torch.where(sims > 1 - eps)
    neighbors = neighbors[0]

    # Maximize least represented concept
    c = balance.get_min_concept()
    point = proto[neigbors][:, c].argmax()
    balance.update(point)

    log_and_keep(point)
    tovisit[neighbors] = 0

# Input: embedding_model, dataset, eps, prototypes
# Embed and cluster the dataset
embeddings = embedding_model(dataset)
per_cluster_embeddings = kmeans(embeddings)
for cluster in per_cluster_embeddings:
  fairdedup(cluster)
'''

'\ndef fairdedup_original(embs, prototypes, eps):\n  # Get similarity with concept prototypes\n  proto = embs @ prototypes.T\n\n  balance = AverageMeter(prototype.shape[0])\n  tovisit = torch.ones(embs.shape[0])\n\n  while tovisit.any():\n    # Find an unvisited neighborhood\n    node = torch.where(tovisit)[0][0]\n    sims = embs[node] @ embs.T\n    neighbors = torch.where(sims > 1 - eps)\n    neighbors = neighbors[0]\n\n    # Maximize least represented concept\n    c = balance.get_min_concept()\n    point = proto[neigbors][:, c].argmax()\n    balance.update(point)\n\n    log_and_keep(point)\n    tovisit[neighbors] = 0\n\n# Input: embedding_model, dataset, eps, prototypes\n# Embed and cluster the dataset\nembeddings = embedding_model(dataset)\nper_cluster_embeddings = kmeans(embeddings)\nfor cluster in per_cluster_embeddings:\n  fairdedup(cluster)\n'